In [150]:
#Загружаем необходимые библиотеки
import pandas as pd
import numpy as np
import plotly.express as px
#Делаем настройки для лучшей читабельности
pd.set_option('display.max_columns',40) #Устанавливаем число отображаемых колонок равное 40
pd.set_option('display.float_format','{:.2f}'.format) #Отображение 2 знаков после запятой

In [151]:
#Загружаем данные и предварительно осматриваем датасет
df = pd.read_csv('data.csv', encoding='1251')
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-11,10:15:35,2173,2004598,2,22577,Розничная реализация,18,45399,ЦИПРОЛЕТ 3МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /Д-...,Катрен г.Химки,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,41.08,51.00,12.00,925.00,200000000492.00,205,1,1.00
1,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,261519,ПЕРЕКИСЬ ВОДОРОДА 3% 100МЛ. №40 Р-Р ФЛ.,Катрен г.Химки,ФЛОРА КАВКАЗА ОАО,1.00,18.61,31.00,3.00,9.00,200010010204.00,205,1,1.00
2,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,460864,СОФЬЯ ГЕЛЬ Д/НОГ ВЕНОТОНИЗ. ТРОКСЕРУТИН ФОРТЕ ...,Катрен г.Химки,КОРОЛЕВФАРМ ООО,1.00,132.69,209.00,20.00,9.00,200010010204.00,205,1,2.00
3,2022-08-11,10:27:46,2174,2004598,2,22577,Розничная реализация,15,172823,СОФЬЯ ГХК КРЕМ Д/ТЕЛА ХОНДРОИТИН+ГЛЮКОЗАМИН 12...,Катрен г.Химки,КОРОЛЕВФАРМ ООО,1.00,133.65,210.00,21.00,9.00,200010010204.00,205,1,3.00
4,2022-08-11,10:33:56,2175,2004598,2,22577,Розничная реализация,18,79056,ГАЛВУС 50МГ. №28 ТАБ. /НОВАРТИС/,Катрен г.Химки,Новартис Фарма АГ,1.00,709.95,787.00,49.00,925.00,200000000492.00,205,1,1.00


In [152]:
#Группируем данные по наименованию товара
#Для агрегации испоьзуем сумму по колонкам 'DR_Kol'- количество проданных товаров и 'DR_CRoz' - сумма выручки без учёта скидок
grouped_df = df.groupby(['DR_NDrugs']).agg({'DR_Kol':sum,'DR_CRoz':sum})

In [153]:
#Считаем вклад каждого товара в общее кол-во проданных товаров
grouped_df['share_kol'] = (grouped_df['DR_Kol']/sum(grouped_df['DR_Kol']))
#Считаем вклад каждого товара в общую сумму проданных товаров
grouped_df['share_roz'] = (grouped_df['DR_CRoz']/sum(grouped_df['DR_CRoz']))

# Сортируем DataFrame по 'share_kol' для расчета кумулятивной суммы по количеству
df_kol_sorted = grouped_df.sort_values('share_kol', ascending=False).copy()
df_kol_sorted['cumsum_kol'] = df_kol_sorted['share_kol'].cumsum()

# Сортируем DataFrame по 'share_roz' для расчета кумулятивной суммы по выручке
df_roz_sorted = grouped_df.sort_values('share_roz', ascending=False).copy()
df_roz_sorted['cumsum_roz'] = df_roz_sorted['share_roz'].cumsum()

# Определяем группу для каждого параметра прямо на отсортированных DataFrame
df_kol_sorted['abc_kol'] = np.where(df_kol_sorted['cumsum_kol'] < 0.8, 'A',
                                   np.where(df_kol_sorted['cumsum_kol'] < 0.95, 'B', 'C'))
df_roz_sorted['abc_roz'] = np.where(df_roz_sorted['cumsum_roz'] < 0.8, 'A',
                                   np.where(df_roz_sorted['cumsum_roz'] < 0.95, 'B', 'C'))

grouped_df['abc_kol'] = df_kol_sorted['abc_kol'] # Pandas автоматически выравнивает по индексу
grouped_df['abc_roz'] = df_roz_sorted['abc_roz'] # Pandas автоматически выравнивает по индексу

In [154]:
#Создаём колонку с итоговым значением по двум группам для каждого товара
grouped_df['abc_group'] = grouped_df['abc_kol'].astype(str) + grouped_df['abc_roz'].astype(str)
final_grouped_df = grouped_df[['DR_Kol','DR_CRoz','share_kol', 'share_roz', 'abc_kol','abc_roz', 'abc_group']]\
                        .sort_values('abc_group')\
                        .reset_index()
final_grouped_df

,DR_NDrugs,DR_Kol,DR_CRoz,share_kol,share_roz,abc_kol,abc_roz,abc_group
0,"ТЕРАФЛЮ ЛИМОН ОТ ГРИППА И ПРОСТУДЫ 22,1Г. №14 ...",7.00,15638.00,0.00,0.01,A,A,AA
1,ДОЛГИТ 5% 50Г. КРЕМ Д/НАРУЖ.ПРИМ. ТУБА /ДОЛОРГИТ/,5.00,722.00,0.00,0.00,A,A,AA
2,ДОЛОБЕНЕ 45Г. №1 ГЕЛЬ Д/НАРУЖ.ПРИМ. ТУБА,2.00,1221.00,0.00,0.00,A,A,AA
3,ВАЛЕРИАНА ЭКСТРАКТ 20МГ. №50 ТАБ. П/О /БОРИСОВ...,24.00,1730.00,0.01,0.00,A,A,AA
4,ОМЕЗ 20МГ. №30 КАПС. /Д-Р РЕДДИ/,10.00,1772.00,0.00,0.00,A,A,AA
...,...,...,...,...,...,...,...,...
1871,КОФЕИН-БЕНЗОАТ НАТРИЯ 100МГ. №10 ТАБ. /ТАТХИМФ...,1.00,93.00,0.00,0.00,C,C,CC
1872,ИМОДИУМ ЭКСПРЕСС 2МГ. №6 ТАБ.ЛИОФ. /ЯНССЕН/,1.00,162.00,0.00,0.00,C,C,CC
1873,КОРИНФАР 10МГ. №100 ТАБ.ПРОЛОНГ. П/П/О /АВД ФА...,1.00,142.00,0.00,0.00,C,C,CC
1874,ЙОДОМАРИН 200 №100 ТАБ. /БЕРЛИН ХЕМИ/,1.00,196.91,0.00,0.00,C,C,CC


In [155]:
#Создание интерактивного графика многомерного ABC - анализа
fig = px.scatter(final_grouped_df,
                 x='share_kol',       # Ось X: Доля в общем количестве
                 y='share_roz',       # Ось Y: Доля в общей выручке
                 color='abc_group',   # Цвет точек по комбинированной ABC-группе
                 size='DR_CRoz',      # Размер точки по общей выручке товара
                 hover_name='DR_NDrugs', # Что отображать при наведении (название товара)
                 hover_data={         # Дополнительные данные для отображения при наведении
                     'DR_Kol': ':.0f',    # Формат для количества (целое число)
                     'DR_CRoz': ':.2f',   # Формат для выручки (2 знака после запятой)
                     'share_kol': ':.2%', # Формат для доли количества (проценты)
                     'share_roz': ':.2%', # Формат для доли выручки (проценты)
                     'abc_kol': True,    # Отобразить группу по количеству
                     'abc_roz': True,    # Отобразить группу по выручке
                     'abc_group': False  # Не повторять эту информацию, она уже в цвете
                 },
                 #Логарифмируем значения для лучшей читабельности
                 log_x=True,
                 log_y=True,
                 title='Интерактивный ABC-анализ: Доля в количестве vs. Доля в выручке',
                 labels={
                     'share_kol': 'Доля в количестве проданных товаров, логарифмированные значения',
                     'share_roz': 'Доля в общей выручке, логарифмированные значения',
                     'abc_group': 'Комбинированная ABC-группа'
                 },
                 color_discrete_map={ # Можно настроить цвета для групп
                     "AA": "darkblue", "AB": "mediumblue", "AC": "lightblue",
                     "BA": "darkgreen", "BB": "mediumseagreen", "BC": "lightgreen",
                     "CA": "darkred", "CB": "indianred", "CC": "lightcoral"
                 }
                )

fig.show()